In [1]:
%load_ext autoreload
%autoreload 3 --print

In [2]:
from kpsn.project.paths import Project, create_model
from kpsn.io import loaders
from kpsn.models import joint
from kpsn import config
from kpsn.models import instantiation, setup, pose, morph, util as model_util
from kpsn import fitting
from kpsn.fitting import em
from kpsn.io import alignment, features
from kpsn.io.armature import Armature
from kpsn.fitting import methods
from kpsn.fitting import scans
import os, sys, shutil
from pprint import pprint
from kpsn import viz
from kpsn.viz import styles
import matplotlib.pyplot as plt

import logging
logging.getLogger().setLevel(logging.INFO)


### project setup

In [3]:
project_dir = '../projects/dataset-refactor'
project = Project(project_dir)

if os.path.exists(project_dir): shutil.rmtree(project_dir)
if not os.path.exists(project_dir):
    loaders.raw_npy.setup_project_config(
        project,
        filepaths = {
            '24wk_m3': '../../../data_explore/data/11_03_22_24wk_m3.npy',
            '5wk_m6': '../../../data_explore/data/10_24_22_5wk_m6.npy'},
        ref_session = '24wk_m3',
        keypoint_names = [
            'shldr', 'back', 'hips', 't_base','t_tip', 'head',
            'l_ear', 'r_ear', 'nose', 'lr_knee', 'lr_foot',
            'rr_knee', 'rr_foot', 'lf_foot', 'rf_foot'],
        keypoint_parents = [
            'back', 'hips', None, 'hips', 'shldr',
            'head', 'head', 'head', 'hips', 'lr_knee',
            'hips', 'rr_knee', 'shldr', 'shldr'],
        exclude_keypoints = ['t_tip'],
        subsample = 100,
        feature_type='pcs',)
    
    setup.setup_base_model_config(project.main_config(), project.base_model_config(),)

plotter, colors = styles.init_nb(project.root() / "plots", style = "vscode_dark")

In [4]:
cfg = config.load_project_config(project.main_config())
dataset = loaders.raw_npy.load(cfg["dataset"])
dataset, cfg = loaders.raw_npy.default_alignment.calibrate(
    dataset, cfg, origin="hips", anterior="head"
)

cfg = features.pcs.calibrate(dataset, cfg)
dataset = features.reduce_to_features(dataset, cfg["features"])
config.save_project_config(project.main_config(), cfg, write_calib=True)

In [5]:
# --- calibration: pose and morph models
cfg = config.load_model_config(project.base_model_config())
cfg["fit"]["n_steps"] = 400

cfg = pose.gmm.calibrate_base_model(dataset, cfg, n_components=3)
cfg = morph.lowrank_affine.calibrate_base_model(dataset, cfg)
cfg = em.calibrate_base_model(dataset, cfg)
config.save_model_config(project.base_model_config(), cfg, write_calib=True)

In [6]:
model_dir, cfg = create_model(
    project,
    "first_run",
    config_overrides={
        "fit.learning_rate": 4e0,
        "fit.n_steps": 40,
    },
)
fit_data = methods.fit(
    project.model("first_run"),
    log_every=1,
)

INFO:root:Initializing morph model
INFO:root:Initializing pose model
INFO:root:Fitting GMM to 360 frames
INFO:root:Adjusting learning rate:4.0 -> 0.005555555555555556
INFO:root:Loading LR schedule: const
INFO:root:Step 0 : loss = 29515.373046875
INFO:root:Step 1 : loss = 29488.6796875
INFO:root:Step 2 : loss = 29478.126953125
INFO:root:Step 3 : loss = 29473.595703125
INFO:root:Step 4 : loss = 29471.3359375
INFO:root:Step 5 : loss = 29469.87890625
INFO:root:Step 6 : loss = 29469.19921875
INFO:root:Step 7 : loss = 29468.87890625
INFO:root:Step 8 : loss = 29468.607421875
INFO:root:Step 9 : loss = 29468.59765625
INFO:root:Step 10 : loss = 29468.505859375
INFO:root:Step 11 : loss = 29468.453125
INFO:root:Step 12 : loss = 29468.345703125
INFO:root:Step 13 : loss = 29468.05078125
INFO:root:Step 14 : loss = 29468.02734375
INFO:root:Step 15 : loss = 29467.759765625
INFO:root:Step 16 : loss = 29467.775390625
INFO:root:Step 17 : loss = 29467.80078125
INFO:root:Step 18 : loss = 29467.796875
INFO:r